In [ ]:
from appgeopy import *
from my_packages import *

In [ ]:
gps_stations = gpd.read_file(
    r".\Shapefiles\extract_incidence_angle_20220609.shp",
    engine="pyogrio",
    read_geometry=False,
)
gps_stations

In [ ]:
fpath = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_葉 大綱\2023_NEW_20240515\20240605_DIFFERENCES\GPS_Differences_NEU_mm_2023.xlsx"
processing_year = fpath.split("_")[-1].split(".")[0]

savefolder = os.path.dirname(fpath)
savename = "GPS_Differences_NEU_LOS_mm_{}.xlsx".format(processing_year)
savepath = os.path.join(savefolder, savename)

available_sheetnames = data_io.get_sheetnames(fpath)

# select_sheet = available_sheetnames[0]
cache = []
for select_sheet in tqdm(available_sheetnames):

    select_gps = gps_stations.query("STATION==@select_sheet")
    
    if len(select_gps)>0:
        inc_angle = select_gps.IncAngle.values[0]
    
        df = pd.read_excel(
            fpath, parse_dates=[0], index_col=[0], sheet_name=select_sheet
        )
        
        LOS_disp = df.apply(
            lambda row: insartools.get_LOS_disp(
                *row, incidence_angle=inc_angle
            ),
            axis=1,
        )
    
        df["LOS(mm)"] = LOS_disp
        df = df.reset_index(drop=False)
        data_io.save_df_to_excel(df_to_save=df, filepath=savepath, sheet_name=select_sheet, verbose=False)
    else:
        cache.append(select_sheet)